In [19]:
# SET CONFIG AND PARAMETERS
with open("setConfig.py") as f:
    code = f.read()
    exec(code)

# Resonator Flux Dependence

In [20]:
class ResonatorFluxDependanceProgram(AveragerProgram):
    
    def ns2cycles(self, ns, gen_ch=None, ro_ch=None):
        return self.us2cycles(ns/1000, gen_ch=gen_ch, ro_ch=ro_ch)

    def initialize(self):
        cfg=self.cfg
        
        probe_ch    = cfg["probe_ch"]        
        feedback_ch = cfg["feedback_ch"]        
        ro_pulse_shape  = cfg["ro_pulse_shape"]
        ro_pulse_length = self.ns2cycles(cfg["ro_pulse_length"], gen_ch=probe_ch)  
        self.declare_gen(ch=probe_ch, 
                         nqz=cfg["probe_ch_nqz"], 
                         mux_freqs=self.cfg['ro_pulse_frequency'], 
                         mux_gains=self.cfg['ro_pulse_gain'])
        self.set_pulse_registers(ch=probe_ch, 
                                 style=ro_pulse_shape, 
                                 length=ro_pulse_length, 
                                 mask=[0,1,2,3])        
        for iCh, ch in enumerate(feedback_ch):
            self.declare_readout(ch=ch, 
                                 freq=self.cfg['acquisition_frequency'][iCh], 
                                 length = self.ns2cycles(self.cfg['acquisition_length'], ro_ch=ch), 
                                 gen_ch=probe_ch)
        self.synci(500)  # give processor some time to configure pulses
    
    def body(self):
        cfg=self.cfg
        probe_ch    = cfg["probe_ch"]
        feedback_ch = cfg["feedback_ch"]
        ro_pulse_start       = self.ns2cycles(cfg["ro_pulse_start"])     
        self.measure(pulse_ch= probe_ch, 
                     adcs = feedback_ch, 
                     adc_trig_offset=self.ns2cycles(cfg["delay_before_acquisition"]), 
                     pins = [0], wait=True, 
                     syncdelay=self.ns2cycles(cfg["relaxation_time"]), 
                     t=ro_pulse_start)

In [21]:
def run_resonator_flux_dependence(gain_range):
    fig = go.Figure()
    experiment_points = 100

    results_matrix1 = np.zeros((len(gain_range), experiment_points))
    results_matrix2 = np.zeros((len(gain_range), experiment_points))
    results_matrix3 = np.zeros((len(gain_range), experiment_points))
    results_matrix4 = np.zeros((len(gain_range), experiment_points))
    for idx, gain in enumerate(tqdm(gain_range)):
        tidac.set_bias(channel_cfg["DAC_bias"][0], bias_value=gain) #+flux_cfg["q1_sweetspot"]
        tidac.set_bias(channel_cfg["DAC_bias"][1], bias_value=gain) #+flux_cfg["q2_sweetspot"]
        tidac.set_bias(channel_cfg["DAC_bias"][2], bias_value=gain) #+flux_cfg["q3_sweetspot"]
        tidac.set_bias(channel_cfg["DAC_bias"][3], bias_value=gain) #+flux_cfg["q4_sweetspot"]
        frequency_range = np.linspace(-2, 2, experiment_points)
        results=[]
        for frequency in frequency_range:
            readout_cfg["ro_pulse_frequency"][0]    = frequency + readout_cfg["freq_original"][0]
            readout_cfg["acquisition_frequency"][0] = frequency + readout_cfg["freq_original"][0]
            readout_cfg["ro_pulse_frequency"][1]    = frequency + readout_cfg["freq_original"][1]
            readout_cfg["acquisition_frequency"][1] = frequency + readout_cfg["freq_original"][1]
            readout_cfg["ro_pulse_frequency"][2]    = frequency + readout_cfg["freq_original"][2]
            readout_cfg["acquisition_frequency"][2] = frequency + readout_cfg["freq_original"][2]
            readout_cfg["ro_pulse_frequency"][3]    = frequency + readout_cfg["freq_original"][3]
            readout_cfg["acquisition_frequency"][3] = frequency + readout_cfg["freq_original"][3]            
            program_cfg = {**channel_cfg,**readout_cfg} 
            program = ResonatorFluxDependanceProgram(soccfg=soc_cfg, cfg=program_cfg)
            results.append(program.acquire(soc, start_src="internal"))
        results = np.transpose(results)

        results_matrix1[idx] = np.abs(results[0][0][0] + 1j*results[0][0][1])
        results_matrix2[idx] = np.abs(results[0][1][0] + 1j*results[0][1][1]) 
        results_matrix3[idx] = np.abs(results[0][2][0] + 1j*results[0][2][1]) 
        results_matrix4[idx] = np.abs(results[0][3][0] + 1j*results[0][3][1])
        
        
    reset_bias()
    return frequency_range, results_matrix1,results_matrix2,results_matrix3,results_matrix4

In [22]:
gain_range =np.linspace(-0.3, 0.1, 40)
freq_range, r1,r2,r3,r4 = run_resonator_flux_dependence(gain_range=gain_range)

100%|██████████| 40/40 [05:25<00:00,  8.14s/it]


In [23]:
# Print results_matrix0,results_matrix1,results_matrix2,results_matrix3
fig = sp.make_subplots(rows=2, cols=2, subplot_titles=("Qubit 6", "Qubit 7", "Qubit 8", "Qubit 9"))
fig.add_trace(go.Heatmap(z=r1, x=freq_range+readout_cfg["freq_original"][0], y=gain_range, colorscale='Viridis', colorbar=dict(title='Signal Magnitude [a.u.]')), row=1, col=1)
fig.add_trace(go.Heatmap(z=r2, x=freq_range+readout_cfg["freq_original"][1], y=gain_range, colorscale='Viridis', colorbar=dict(title='Signal Magnitude [a.u.]')), row=1, col=2)    
fig.add_trace(go.Heatmap(z=r3,  x=freq_range+readout_cfg["freq_original"][2], y=gain_range, colorscale='Viridis', colorbar=dict(title='Signal Magnitude [a.u.]')), row=2, col=1)    
fig.add_trace(go.Heatmap(z=r4,  x=freq_range+readout_cfg["freq_original"][3], y=gain_range, colorscale='Viridis', colorbar=dict(title='Signal Magnitude [a.u.]')), row=2, col=2)    
#fig.add_trace(go.Heatmap(z=results_matrix, x=cf, y=gain_range, colorscale='Viridis', colorbar=dict(title='Signal Magnitude [a.u.]')))
# Update x-axis and y-axis labels
fig.update_xaxes(title_text="Frequency (MHz)", tickangle=-45)
fig.update_yaxes(title_text="Gain")
fig.update_layout(title='Resonator Flux Dependence', 
                  height = 1000, width = 1000,
                  title_x= 0.5,
)
fig.show()

# Set fluxes to zero
reset_bias()

In [6]:
gain_range


array([-0.5       , -0.48989899, -0.47979798, -0.46969697, -0.45959596,
       -0.44949495, -0.43939394, -0.42929293, -0.41919192, -0.40909091,
       -0.3989899 , -0.38888889, -0.37878788, -0.36868687, -0.35858586,
       -0.34848485, -0.33838384, -0.32828283, -0.31818182, -0.30808081,
       -0.2979798 , -0.28787879, -0.27777778, -0.26767677, -0.25757576,
       -0.24747475, -0.23737374, -0.22727273, -0.21717172, -0.20707071,
       -0.1969697 , -0.18686869, -0.17676768, -0.16666667, -0.15656566,
       -0.14646465, -0.13636364, -0.12626263, -0.11616162, -0.10606061,
       -0.0959596 , -0.08585859, -0.07575758, -0.06565657, -0.05555556,
       -0.04545455, -0.03535354, -0.02525253, -0.01515152, -0.00505051,
        0.00505051,  0.01515152,  0.02525253,  0.03535354,  0.04545455,
        0.05555556,  0.06565657,  0.07575758,  0.08585859,  0.0959596 ,
        0.10606061,  0.11616162,  0.12626263,  0.13636364,  0.14646465,
        0.15656566,  0.16666667,  0.17676768,  0.18686869,  0.19

In [9]:
print(np.linspace(-.1, .4, 50))

[-0.1        -0.08979592 -0.07959184 -0.06938776 -0.05918367 -0.04897959
 -0.03877551 -0.02857143 -0.01836735 -0.00816327  0.00204082  0.0122449
  0.02244898  0.03265306  0.04285714  0.05306122  0.06326531  0.07346939
  0.08367347  0.09387755  0.10408163  0.11428571  0.1244898   0.13469388
  0.14489796  0.15510204  0.16530612  0.1755102   0.18571429  0.19591837
  0.20612245  0.21632653  0.22653061  0.23673469  0.24693878  0.25714286
  0.26734694  0.27755102  0.2877551   0.29795918  0.30816327  0.31836735
  0.32857143  0.33877551  0.34897959  0.35918367  0.36938776  0.37959184
  0.38979592  0.4       ]
